In [1]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated, partition

In [2]:
X_train = MNIST.images()
y_train = onehotbatch(MNIST.labels(), 0:9)
X_test = MNIST.images(:test)
y_test = onehotbatch(MNIST.labels(:test), 0:9)

10×10000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false   true  false  false     false  false  false  false  false
 false   true  false  false  false      true  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false  false  false   true     false  false   true  false  false
 false  false  false  false  false  …  false  false  false   true  false
 false  false  false  false  false     false  false  false  false   true
  true  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false

In [3]:
# Partition into batches of size 1,000
training_data = [(cat(4, float.(X_train[i])...), y_train[:,i])
         for i in partition(1:60_000, 100)]

# X_train = cat(4, float.(X_train)...)
X_test = cat(4, float.(X_test)...)

28×28×1×10000 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.329412     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.870588     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.262745     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 ⋮  

In [16]:
model = Chain(
            #28 * 28
          Conv2D((3, 3), 1=>32, relu),
            #26 * 26 * 32   
          x -> maxpool2d(x, 2),
            #13 * 13 * 32   
          Conv2D((3, 3), 32=>64, relu),
            #11 * 11 * 64
          x -> maxpool2d(x, 2),
            #5 * 5 * 64
          x -> reshape(x, :, size(x, 4)),
          Dense(1600, 128, relu),
          Dense(128, 10), 
          softmax)

Chain(Conv2D((3, 3), 1=>32, NNlib.relu), #21, Conv2D((3, 3), 32=>64, NNlib.relu), #22, #23, Dense(1600, 128, NNlib.relu), Dense(128, 10), NNlib.softmax)

In [17]:
# Use cross entropy loss here
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(argmax(model(x)) .== argmax(y))

accuracy (generic function with 1 method)

In [18]:
model(training_data[1][1])

Tracked 10×100 Array{Float64,2}:
 0.100261   0.100104   0.100141   …  0.100142   0.100229   0.100148 
 0.100054   0.0999251  0.099972      0.0998896  0.100056   0.0998122
 0.0997385  0.0999375  0.099703      0.0997396  0.0998294  0.0997997
 0.0999268  0.100244   0.100141      0.0999377  0.099821   0.100013 
 0.0997632  0.0999284  0.099768      0.099831   0.0997495  0.0999184
 0.0995322  0.0998416  0.0998644  …  0.10008    0.0997491  0.0999026
 0.100143   0.100083   0.100041      0.100047   0.100057   0.100111 
 0.100191   0.0999573  0.0999597     0.100085   0.100203   0.100168 
 0.100144   0.0999102  0.099934      0.100124   0.10014    0.10007  
 0.100247   0.100069   0.100476      0.100123   0.100166   0.100057 

In [ ]:
# Callback during the training
function eval_cb()
    
#     println("Training loss: $(loss(X_train, y_train)), acc: $(accuracy(X_train, y_train))")
    println("Test loss: $(loss(X_test, y_test)), acc: $(accuracy(X_test, y_test))")
    println()
end

optimizer = ADAM(params(model))

Flux.train!(loss, training_data, optimizer, cb = throttle(eval_cb, 10))

Test loss: param(2.30067), acc: 0.1905



In [7]:
println("Optimization Finished!")
@printf "Model accuracy on test set: %.3f\n" accuracy(X_test, y_test)

Optimization Finished!


LoadError: [91mMethodError: no method matching conv2d(::Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}, ::TrackedArray{…,Array{Float64,4}}; stride=1, padding=0)[0m
Closest candidates are:
  conv2d([91m::Flux.Tracker.TrackedArray{#s7,4,A} where A where #s7[39m, ::Flux.Tracker.TrackedArray{#s6,4,A} where A where #s6; stride, padding) at /Users/hitmann47/.julia/v0.6/Flux/src/tracker/lib.jl:142
  conv2d([91m::AbstractArray{#s7,4} where #s7[39m, ::Flux.Tracker.TrackedArray{#s6,4,A} where A where #s6; stride, padding) at /Users/hitmann47/.julia/v0.6/Flux/src/tracker/lib.jl:144
  conv2d([91m::Flux.Tracker.TrackedArray{#s7,4,A} where A where #s7[39m, ::AbstractArray{#s6,4} where #s6; stride, padding) at /Users/hitmann47/.julia/v0.6/Flux/src/tracker/lib.jl:146[39m